<a href="https://colab.research.google.com/github/davidwan08/Feature-Engineering-Bike-Share-Rentals/blob/main/Coding_Dojo_Feature_Engineering_Bike_Share_Rentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
# Assignment: Feature Engineering Exercise (Core)

# Name: David Wan

# Date: 05/25/2022

# **Assignment Overview**

In this exercise you will be working with data about bike share rentals.  You can download the data here.

https://docs.google.com/spreadsheets/d/e/2PACX-1vROUXPkYUkX-2W7JbJ0-oNKaXzpg4NtmU9IeWEY6yFKm32ZEJOpRh_soHD4BeIcuHjYik3SEoXmkgwj/pub?output=csv

Your task is to engineer some new features to try to improve a mode's ability to predict the total number of bike share rentals during a given hour of the day.

1. Import the data the drop the 'casual' and 'registered' columns.  These are redundant with your target, 'count'.  



2. Transform the 'datetime' column into a datetime type and use it to create 3 new columns in the data frame containing the:

  1. Name of the Month
  2. Name of the Day of the Week
  3. Hour of the Day

   - Make sure all 3 new columns are 'object' datatype so they can be one-hot encoded later.  

   - Drop the 'datetime' and 'season' columns.  These are now redundant.



3. The temperatures in the 'temp' and 'atemp' column are in Celsius.  Use `.apply()` to convert them to Fahrenheit.



4. Create a new column, 'temp_variance' that is the difference between 'temp' and 'atemp'.  Drop the 'atemp' column.



**Optional: **

Use a predictive model of your choice and try to predict the 'count' of hourly bikeshare users with both the original features and the engineered feature set you just created.  

Remember to drop the 'casual' and 'registered' columns from both versions before modeling.

Did these feature engineering choices improve your ability to predict the 'count'?

# **Import Libraries**

In [53]:
# Import the necessary libraries.
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.ensemble import BaggingRegressor

# **Data Loading**

- I will use the above link to access the data.

In [54]:
# Load the dataset into the dataframe.
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vROUXPkYUkX-2W7JbJ0-oNKaXzpg4NtmU9IeWEY6yFKm32ZEJOpRh_soHD4BeIcuHjYik3SEoXmkgwj/pub?output=csv')
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 0:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 1:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 2:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 3:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 4:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [55]:
# Get an overview of the dataset.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


In [56]:
# Check for duplicated values.
df.duplicated().sum()

0

In [57]:
# Check for missing values.
df.isna().sum()

datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
casual        0
registered    0
count         0
dtype: int64

# **Dropping 'casual' and 'registered' Columns**

- Per the assignment instructions, I will drop the 'casual' and 'registered' columns, since these columns are redundant with respect to the target column 'count'.

In [58]:
# Drop the casual and registered columns from the dataset.
df.drop(columns = ['casual','registered'], inplace = True)
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
0,2011-01-01 0:00:00,1,0,0,1,9.84,14.395,81,0.0,16
1,2011-01-01 1:00:00,1,0,0,1,9.02,13.635,80,0.0,40
2,2011-01-01 2:00:00,1,0,0,1,9.02,13.635,80,0.0,32
3,2011-01-01 3:00:00,1,0,0,1,9.84,14.395,75,0.0,13
4,2011-01-01 4:00:00,1,0,0,1,9.84,14.395,75,0.0,1


# **Dataset Copy for Modeling Purposes**

- For the purpose of modeling, I will create a copy of the dataset and rename as original.

In [59]:
# Create a copy of the dataset as the original.
df_original = df.copy(deep = True)
df_original.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
0,2011-01-01 0:00:00,1,0,0,1,9.84,14.395,81,0.0,16
1,2011-01-01 1:00:00,1,0,0,1,9.02,13.635,80,0.0,40
2,2011-01-01 2:00:00,1,0,0,1,9.02,13.635,80,0.0,32
3,2011-01-01 3:00:00,1,0,0,1,9.84,14.395,75,0.0,13
4,2011-01-01 4:00:00,1,0,0,1,9.84,14.395,75,0.0,1


# **'datetime' Column Transformation**

- Since the datetime column was loaded as an object type, we would need to change the data type of datetime to datetime.

In [60]:
# Change the data type of the datetime column to datetime.
df['datetime'] = pd.to_datetime(df['datetime'])
print(df['datetime'].dtype)
df.info()

datetime64[ns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    10886 non-null  datetime64[ns]
 1   season      10886 non-null  int64         
 2   holiday     10886 non-null  int64         
 3   workingday  10886 non-null  int64         
 4   weather     10886 non-null  int64         
 5   temp        10886 non-null  float64       
 6   atemp       10886 non-null  float64       
 7   humidity    10886 non-null  int64         
 8   windspeed   10886 non-null  float64       
 9   count       10886 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(6)
memory usage: 850.6 KB


- Based on the instructions, the 3 new columns we will need are the month name, day name of the week, and hour of the day.

In [61]:
# Extract the aforementioned information from the datetime column, then create new columns.
df['monthname'] = df['datetime'].dt.month_name()
df['weekdayname'] = df['datetime'].dt.day_name()
df['hour'] = df['datetime'].dt.hour
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,monthname,weekdayname,hour
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,16,January,Saturday,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,40,January,Saturday,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,32,January,Saturday,2
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,13,January,Saturday,3
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,1,January,Saturday,4


In [62]:
# Get an overview of the dataset.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   datetime     10886 non-null  datetime64[ns]
 1   season       10886 non-null  int64         
 2   holiday      10886 non-null  int64         
 3   workingday   10886 non-null  int64         
 4   weather      10886 non-null  int64         
 5   temp         10886 non-null  float64       
 6   atemp        10886 non-null  float64       
 7   humidity     10886 non-null  int64         
 8   windspeed    10886 non-null  float64       
 9   count        10886 non-null  int64         
 10  monthname    10886 non-null  object        
 11  weekdayname  10886 non-null  object        
 12  hour         10886 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(7), object(2)
memory usage: 1.1+ MB


- We want the 3 new columns created to have object datatype to allow one hot encoding later on, so let's change the hour datatype here.

In [63]:
# Change the datatype of the hour column to object.
df['hour'] = df['hour'].astype('object')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   datetime     10886 non-null  datetime64[ns]
 1   season       10886 non-null  int64         
 2   holiday      10886 non-null  int64         
 3   workingday   10886 non-null  int64         
 4   weather      10886 non-null  int64         
 5   temp         10886 non-null  float64       
 6   atemp        10886 non-null  float64       
 7   humidity     10886 non-null  int64         
 8   windspeed    10886 non-null  float64       
 9   count        10886 non-null  int64         
 10  monthname    10886 non-null  object        
 11  weekdayname  10886 non-null  object        
 12  hour         10886 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(6), object(3)
memory usage: 1.1+ MB


- Now that we have extracted the necessary information from the datetime column, we can go ahead and drop the datetime and season columns.

In [64]:
# Drop the datetime and season columns from the dataset.
df.drop(columns = ['datetime','season'], inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   holiday      10886 non-null  int64  
 1   workingday   10886 non-null  int64  
 2   weather      10886 non-null  int64  
 3   temp         10886 non-null  float64
 4   atemp        10886 non-null  float64
 5   humidity     10886 non-null  int64  
 6   windspeed    10886 non-null  float64
 7   count        10886 non-null  int64  
 8   monthname    10886 non-null  object 
 9   weekdayname  10886 non-null  object 
 10  hour         10886 non-null  object 
dtypes: float64(3), int64(5), object(3)
memory usage: 935.6+ KB


# **'temp' and 'atemp' Column Transformation**

- Since the 'temp' and 'atemp' columns are in Celsius units, we need to convert them both to Fahrenheit units.
- Let's first create a function to allow this conversion.
- Source: https://www.javatpoint.com/python-convert-celsius-to-fahrenheit

In [65]:
# Create a function that will allow the temperature conversion.
def C_to_F(temp):
  converted_temp = (temp * 1.8) + 32
  return converted_temp

- Next, we will use the .apply() method to convert these columns.

In [66]:
# Transform the temp and atemp columns into Fahrenheit units.
df['temp'] = df['temp'].apply(C_to_F)
df['atemp'] = df['atemp'].apply(C_to_F)
df.head()

,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,monthname,weekdayname,hour
0,0,0,1,49.712,57.911,81,0.0,16,January,Saturday,0
1,0,0,1,48.236,56.543,80,0.0,40,January,Saturday,1
2,0,0,1,48.236,56.543,80,0.0,32,January,Saturday,2
3,0,0,1,49.712,57.911,75,0.0,13,January,Saturday,3
4,0,0,1,49.712,57.911,75,0.0,1,January,Saturday,4


# **'temp_variance' Column Creation**

- We will define a new column called 'temp_variance' that will take the difference between 'temp' and 'atemp.'

In [67]:
# Create the temp_variance column using temp and atemp differences.
df['temp_variance'] = df['temp'] - df['atemp']
df.head()

,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,monthname,weekdayname,hour,temp_variance
0,0,0,1,49.712,57.911,81,0.0,16,January,Saturday,0,-8.199
1,0,0,1,48.236,56.543,80,0.0,40,January,Saturday,1,-8.307
2,0,0,1,48.236,56.543,80,0.0,32,January,Saturday,2,-8.307
3,0,0,1,49.712,57.911,75,0.0,13,January,Saturday,3,-8.199
4,0,0,1,49.712,57.911,75,0.0,1,January,Saturday,4,-8.199


- Now that the new column has been created, we can safely drop the atemp column from the dataset.

In [68]:
# Drop the atemp column from the dataset.
df.drop(columns = 'atemp', inplace = True)
df.head()

,holiday,workingday,weather,temp,humidity,windspeed,count,monthname,weekdayname,hour,temp_variance
0,0,0,1,49.712,81,0.0,16,January,Saturday,0,-8.199
1,0,0,1,48.236,80,0.0,40,January,Saturday,1,-8.307
2,0,0,1,48.236,80,0.0,32,January,Saturday,2,-8.307
3,0,0,1,49.712,75,0.0,13,January,Saturday,3,-8.199
4,0,0,1,49.712,75,0.0,1,January,Saturday,4,-8.199


In [69]:
# Get an overview of the dataset.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   holiday        10886 non-null  int64  
 1   workingday     10886 non-null  int64  
 2   weather        10886 non-null  int64  
 3   temp           10886 non-null  float64
 4   humidity       10886 non-null  int64  
 5   windspeed      10886 non-null  float64
 6   count          10886 non-null  int64  
 7   monthname      10886 non-null  object 
 8   weekdayname    10886 non-null  object 
 9   hour           10886 non-null  object 
 10  temp_variance  10886 non-null  float64
dtypes: float64(3), int64(5), object(3)
memory usage: 935.6+ KB


- The dataset is now ready for modeling!

# **Optional - Predictive Modeling**

- I will use bagged regression model to perform the modeling process for both the original and engineered datasets.
- Let's see how feature engineering will make a difference in modeling performance.

## **Dataset Feature Selection**

In [70]:
# Assign the features and target columns for both datasets.
X = df.drop(columns = 'count')
y = df['count']
X_orig = df_original.drop(columns = 'count')
y_orig = df_original['count']

## **Train Test Split**

In [71]:
# Perform the train test split of both datasets using random state 42.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_orig_train, X_orig_test, y_orig_train, y_orig_test = train_test_split(X_orig, y_orig, random_state=42)

## **Preprocessing Pipeline**

In [72]:
# Create the column selectors.
cat_selector = make_column_selector(dtype_include = 'object')
num_selector = make_column_selector(dtype_include = 'number')
# Instantiate the necessary classes.
scaler = StandardScaler()
ohe = OneHotEncoder(handle_unknown = 'ignore', sparse = False)
bagreg = BaggingRegressor(random_state = 42)

In [73]:
# Create the tuples necessary for column transformation.
cat_tuple = (ohe, cat_selector)
num_tuple = (scaler, num_selector)
# Create the preprocessing pipeline.
preprocessor = make_column_transformer(cat_tuple, num_tuple, remainder = 'drop')

## **Bagged Regression Modeling**

In [74]:
# Create the pipeline for logistic regression.
bagreg_pipe = make_pipeline(preprocessor, bagreg)

In [75]:
%%time
# Make predictions on the engineered dataset.
# Then print the train and test scores for the engineered model.
bagreg_pipe.fit(X_train, y_train)
X_train_preds_eng = bagreg_pipe.predict(X_train)
X_test_preds_eng = bagreg_pipe.predict(X_test)
print(bagreg_pipe.score(X_train, y_train))
print(bagreg_pipe.score(X_test, y_test))

0.9657860971930354
0.8208656457386024
CPU times: user 610 ms, sys: 62 ms, total: 672 ms
Wall time: 666 ms


In [76]:
%%time
# Make predictions on the original dataset.
# Then print the train and test scores for the original model.
bagreg_pipe.fit(X_orig_train, y_orig_train)
X_train_preds_orig = bagreg_pipe.predict(X_orig_train)
X_test_preds_orig = bagreg_pipe.predict(X_orig_test)
print(bagreg_pipe.score(X_orig_train, y_orig_train))
print(bagreg_pipe.score(X_orig_test, y_orig_test))

0.8737644072812063
0.23145524691184738
CPU times: user 27.4 s, sys: 184 ms, total: 27.6 s
Wall time: 28.9 s


- Based on the modeling scores I obtained from both the original and engineered datasets, the engineered dataset came up with better test scores than the original dataset.
- Not only did the engineered dataset perform the fitting process more properly, but it also did so with significantly less time than the original.
- This showed that feature engineering was crucial in helping expedite the modeling process of the data.